In [4]:
import numpy as np
import os
import matplotlib.pyplot as plt
from importlib import reload

# Tests of new features

Use this notebook to test some modifications on the classes without the need to reinstall the package 

In [5]:
import sys
sys.path.insert(0,'/home/marco/Applications/MPPI/mppi/InputFiles/')
sys.path.insert(0,'/home/marco/Applications/MPPI/mppi/Calculators/')
sys.path.insert(0,'/home/marco/Applications/MPPI/mppi/Datasets/')
import InputFiles as I, Calculators as C, Datasets as D

In [6]:
reload(I)
input = I.PwIn()

In [315]:
k = 6
ecut = 80
a0 = 1.42
planeSep  = 10.0
A = [0.,0.,0.]
B = [0.,a0,0.]
a1 = [a0*np.sqrt(3),0.,0.]
a2 = [a0*np.sqrt(3)/2,a0*3./2,0.]
a3 = [0.,0.,planeSep]

input.set_calculation('nscf')
input.system['nbnd'] = 12
input.set_pseudo_dir('../pseudos')
input.system['ibrav'] = 0
input.cell_parameters =[a1,a2,a3]
input.set_atoms_type(1)
input.set_atoms_number(2)
input.set_atoms_position(pos_type='angstrom',pos_list=[['C',A],['C',B]])
input.atypes = {'C': [12.011,'C_pbe-20082014.UPF']}
input.set_convergence_thr(1e-8)
#prefix = 'k'+str(k)+'_ecut'+str(ecut)
#input.set_prefix(prefix)
input.set_kpoints(ktype='automatic',kpoints=[k,k,1])
input.set_energy_cutoff(ecut)

print(input)

&control
           verbosity = 'high'
          pseudo_dir = '../pseudos'
              prefix = 'ecut:100,k:6'
         calculation = 'nscf'
/&end
&system
               ibrav = 0
                ntyp = 1
                 nat = 2
             ecutwfc = 80
                nbnd = 12
/&end
&electrons
            conv_thr = 1e-08
/&end
ATOMIC_SPECIES
   C   12.011   C_pbe-20082014.UPF
ATOMIC_POSITIONS { angstrom }
  C   0.0000000000   0.0000000000   0.0000000000
  C   0.0000000000   1.4200000000   0.0000000000
K_POINTS { automatic }
  6  6  1  0  0  0
CELL_PARAMETERS angstrom
  2.4595121467   0.0000000000   0.0000000000 
  1.2297560734   2.1300000000   0.0000000000 
  0.0000000000   0.0000000000  10.0000000000 



## pre_processing for dataset

In some cases it can be useful to perform some preliminary operations before running a computation. E.g. copy of the scf save folder in the appropriate location before starting with a nscf computation or preparing the SAVE folder before a Yambo computation.

For this reason we can introduce a pre_processing function 

In [297]:
reload(D)

<module 'Datasets' from '/home/marco/Applications/MPPI/mppi/Datasets/Datasets.py'>

In [316]:
code = C.QeCalculator(omp=1,mpi_run='mpirun -np 4',executable='pw.x',verbose=True,skip=False)

Initialize a qe calculator with command OMP_NUM_THREADS=1 mpirun -np 4 pw.x


In [317]:
study = D.Dataset(label='prova',run_dir='nscf',pre_processing='nscf',source_dir='gs_run_dir')

In [318]:
study.pre_processing

'nscf'

In [319]:
study.kwargs

{'pre_processing': 'nscf', 'source_dir': 'gs_run_dir'}

In [320]:
kpoints = [3,6]

In [321]:
for k in kpoints:
    idd = {'k' : k, 'ecut' : 100} 
    input.set_prefix(D.name_from_id(idd))
    input.set_kpoints(ktype='automatic',kpoints=[k,k,1])
    input.set_energy_cutoff(100)
    study.append_run(id=idd,calculator=code,input=input)

In [322]:
print(study.runs[1])

&control
           verbosity = 'high'
          pseudo_dir = '../pseudos'
              prefix = 'ecut:100,k:6'
         calculation = 'nscf'
/&end
&system
               ibrav = 0
                ntyp = 1
                 nat = 2
             ecutwfc = 100
                nbnd = 12
/&end
&electrons
            conv_thr = 1e-08
/&end
ATOMIC_SPECIES
   C   12.011   C_pbe-20082014.UPF
ATOMIC_POSITIONS { angstrom }
  C   0.0000000000   0.0000000000   0.0000000000
  C   0.0000000000   1.4200000000   0.0000000000
K_POINTS { automatic }
  6  6  1  0  0  0
CELL_PARAMETERS angstrom
  2.4595121467   0.0000000000   0.0000000000 
  1.2297560734   2.1300000000   0.0000000000 
  0.0000000000   0.0000000000  10.0000000000 



In [324]:
study.run()

nsct save folder already exsists. Save folder NOT COPIED
nsct save folder already exsists. Save folder NOT COPIED
execute : cd nscf ; OMP_NUM_THREADS=1 mpirun -np 4 pw.x -inp ecut:100,k:3.in > ecut:100,k:3.log
execute : cd nscf ; OMP_NUM_THREADS=1 mpirun -np 4 pw.x -inp ecut:100,k:6.in > ecut:100,k:6.log
